In [2]:

import pandas as pd

dfd = pd.read_csv('../data/train.csv', na_values=('#NUM!', '#DIV/0!'))

In [3]:
test = pd.read_csv('../data/test.csv', na_values=('#NUM!', '#DIV/0!'))
samplesubmission = pd.read_csv('../data/sample_submission.csv')

In [4]:
columns_to_convert = ['lnFormaldehyde_tropospheric_HCHO_column_number_density_amf',
                      'logCarbonMonoxide_H2O_column_number_density',
                      'Cloud_surface_albedo']

for column in columns_to_convert:
    dfd[column] = pd.to_numeric(dfd[column], errors='coerce')

# Now, check the data types of each column
print(dfd.dtypes)

KeyError: 'lnFormaldehyde_tropospheric_HCHO_column_number_density_amf'

In [39]:
import pandas as pd

# Assuming dfd is the DataFrame you have and contains the required columns.

# Calculate the medians
median_value1 = dfd['lnFormaldehyde_tropospheric_HCHO_column_number_density_amf'].median()
median_value2 = dfd['logCarbonMonoxide_H2O_column_number_density'].median()
median_value3 = dfd['Cloud_surface_albedo'].median()

# Print the median values
print("Median of lnFormaldehyde_tropospheric_HCHO_column_number_density_amf:", median_value1)
print("Median of logCarbonMonoxide_H2O_column_number_density:", median_value2)
print("Median of Cloud_surface_albedo:", median_value3)

Median of lnFormaldehyde_tropospheric_HCHO_column_number_density_amf: 0.192904627
Median of logCarbonMonoxide_H2O_column_number_density: 7.618451077
Median of Cloud_surface_albedo: 0.272747239


In [40]:
median_value11 = test['lnFormaldehyde_tropospheric_HCHO_column_number_density_amf'].median()
median_value22 = test['logCarbonMonoxide_H2O_column_number_density'].median()
median_value33 = test['Cloud_surface_albedo'].median()

# Print the median values
print("Median of lnFormaldehyde_tropospheric_HCHO_column_number_density_amf:", median_value11)
print("Median of logCarbonMonoxide_H2O_column_number_density:", median_value22)
print("Median of Cloud_surface_albedo:", median_value33)

Median of lnFormaldehyde_tropospheric_HCHO_column_number_density_amf: 0.171699825
Median of logCarbonMonoxide_H2O_column_number_density: 7.4517882185
Median of Cloud_surface_albedo: 0.25727885


In [41]:
nan_values = dfd['lnFormaldehyde_tropospheric_HCHO_column_number_density_amf'].isna()
# NaN 값을 1.25413으로 대체
dfd.loc[nan_values, 'lnFormaldehyde_tropospheric_HCHO_column_number_density_amf'] = median_value1

In [42]:
nan_values11 = test['lnFormaldehyde_tropospheric_HCHO_column_number_density_amf'].isna()
# NaN 값을 1.25413으로 대체
test.loc[nan_values11, 'lnFormaldehyde_tropospheric_HCHO_column_number_density_amf'] = median_value11

In [43]:
nan_values2 = dfd['logCarbonMonoxide_H2O_column_number_density'].isna()
# NaN 값을 1.25413으로 대체
dfd.loc[nan_values2, 'logCarbonMonoxide_H2O_column_number_density'] = median_value2

In [44]:
nan_values22 = test['logCarbonMonoxide_H2O_column_number_density'].isna()
# NaN 값을 1.25413으로 대체
test.loc[nan_values22, 'logCarbonMonoxide_H2O_column_number_density'] = median_value22

In [45]:
nan_values3 = dfd['Cloud_surface_albedo'].isna()
# NaN 값을 1.25413으로 대체
dfd.loc[nan_values3, 'Cloud_surface_albedo'] = median_value3

In [46]:
nan_values33 = test['Cloud_surface_albedo'].isna()
# NaN 값을 1.25413으로 대체
test.loc[nan_values33, 'Cloud_surface_albedo'] = median_value33

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
import re

from category_encoders import OneHotEncoder, MEstimateEncoder, GLMMEncoder, OrdinalEncoder, CatBoostEncoder
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, KFold, TimeSeriesSplit
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor, VotingRegressor, StackingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import HuberRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.preprocessing import FunctionTransformer, StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.calibration import CalibratedClassifierCV
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

sns.set_theme(style = 'white', palette = 'colorblind')
pal = sns.color_palette('colorblind')

pd.set_option('display.max_rows', 100)

In [48]:
X = dfd.copy()
y = X.pop('lnemission')

seed = 42
k = LeaveOneGroupOut()

np.random.seed(seed)

In [55]:
def feature_dropper(x):
    return x[['lnFormaldehyde_tropospheric_HCHO_column_number_density_amf', 'longitude', 'week_no', 'month', 'is_covid','logCarbonMonoxide_H2O_column_number_density']]

FeatureDropper = FunctionTransformer(feature_dropper)

In [62]:
def date_processor(x):
    x_copy = x.copy()
    year_week = pd.to_datetime((x_copy.year * 100 + x_copy.week_no).astype('str') + '0', format = '%Y%W%w')
    x_copy['month'] = year_week.dt.month
    x_copy['is_covid'] = (x_copy.year == 2020) & (x_copy.month > 2)
    return x_copy
DateProcessor = FunctionTransformer(date_processor)

In [57]:
def cross_val_score(model, cv = k, label = ''):
    
    X = dfd.copy()
    y = X.pop('lnemission')
    
    #initiate prediction arrays and score lists
    val_predictions = np.zeros((len(dfd)))
    #train_predictions = np.zeros((len(train)))
    train_scores, val_scores = [], []
    
    #training model, predicting prognosis probability, and evaluating log loss
    for fold, (dfd_idx, val_idx) in enumerate(cv.split(X, y, X.year)):
        
        #define train set
        X_train = X.iloc[dfd_idx]
        y_train = y.iloc[dfd_idx]
        
        #define validation set
        X_val = X.iloc[val_idx]
        y_val = y.iloc[val_idx]
        
        #train model
        model.fit(X_train, y_train)
        
        #make predictions
        train_preds = model.predict(X_train)
        val_preds = model.predict(X_val)
                  
        val_predictions[val_idx] += val_preds
        
        #evaluate model for a fold
        train_score = mean_squared_error(y_train, train_preds, squared = False)
        val_score = mean_squared_error(y_val, val_preds, squared = False)
        
        #append model score for a fold to list
        train_scores.append(train_score)
        val_scores.append(val_score)
    
    print(f'Val Score: {np.mean(val_scores):.5f} ± {np.std(val_scores):.5f} | Train Score: {np.mean(train_scores):.5f} ± {np.std(train_scores):.5f} | {label}')
    
    return val_scores, val_predictions

In [58]:
score_list, oof_list = pd.DataFrame(), pd.DataFrame()

models = [
    ('ridge', Ridge(random_state = seed)),
    ('rf', RandomForestRegressor(random_state = seed)),
    ('et', ExtraTreesRegressor(random_state = seed)),
    ('xgb', XGBRegressor(random_state = seed)),
    ('lgb', LGBMRegressor(random_state = seed)),
    ('cb', CatBoostRegressor(random_state = seed, verbose = 0)),
    ('gb', GradientBoostingRegressor(random_state = seed)),
    ('hgb', HistGradientBoostingRegressor(random_state = seed))
]


In [59]:
for (label, model) in models:
     score_list[label], oof_list[label] = cross_val_score(
         make_pipeline(DateProcessor, FeatureDropper, model),
         label = label,
     )

Val Score: 1.91815 ± 0.02285 | Train Score: 1.91012 ± 0.01090 | ridge
Val Score: 0.49656 ± 0.02053 | Train Score: 0.17428 ± 0.00220 | rf
Val Score: 0.57560 ± 0.01042 | Train Score: 0.02754 ± 0.00794 | et
Val Score: 0.56313 ± 0.02004 | Train Score: 0.49042 ± 0.00691 | xgb
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006279 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 833
[LightGBM] [Info] Number of data points in the train set: 52682, number of used features: 6
[LightGBM] [Info] Start training from score 3.199796
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002969 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 831
[LightGBM] [Info] Number of data points in the train set: 52682, number of used features: 5
[LightGBM] [Info] Start training from score 3.301991
[LightGBM] [Warning] Auto-choosing col-wis

In [60]:
model = make_pipeline(DateProcessor, FeatureDropper, RandomForestRegressor(random_state = seed))

model.fit(X, y)
prediction = model.predict(test)


In [61]:
test['lnemission'] = prediction
test.to_csv('submission5.csv', index = False)